<a href="https://colab.research.google.com/github/andreidm92/Agents_in_code/blob/main/practice/Lesson_02_qna_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Day 2 — Sales Manager: Q&A Bot

### Теория
Сегодня мы изучаем:
- **LLM Wrappers** — интерфейс к языковым моделям (например, OpenAI, Claude).
- **Prompt Templates** — шаблоны сообщений с переменными.
- **Conditional Edges (LangGraph)** — условные переходы на основе результата.

### Пример
```python
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model_name="gpt-4o")
prompt = ChatPromptTemplate.from_messages([
    ("system", "Ты — ассистент по продукту."),
    ("human", "Вопрос: {question}")
])
chain = prompt | llm
print(chain.invoke({"question": "Какие у нас есть тарифы?"}))
```


In [ ]:
# Установка библиотек
!pip install langchain openai
!pip install langchain_community
!pip install langchain_openai
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 9.5 MB/s eta 0:00:00


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


In [ ]:
import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Вставь OpenAI API ключ: ")

Вставь OpenAI API ключ: ··········


In [ ]:
# TODO: Загрузка файла с Q&A (можно заменить на словарь вручную)
faq = {
    "Какие тарифы?": "У нас есть три тарифа: Базовый, Продвинутый и Премиум.",
    "Как отменить подписку?": "Вы можете отменить подписку в настройках аккаунта."
}


In [ ]:
# TODO: Настроим шаблон и LLM
prompt = ChatPromptTemplate.from_messages([
    ("system", "Ты — эксперт по продукту."),
    ("human", "Вопрос: {question}")
])

llm = ChatOpenAI(model_name="gpt-4o")


In [ ]:

# TODO: Функция ответа с проверкой длины
#❓ Где Conditional Edge в текущем ноутбуке?
#На самом деле — у нас упрощённая имитация Conditional Edge:

def ask_bot(question):
    response = llm.invoke(prompt.format_messages(question=question))
    answer = response.content
    if len(answer.split()) < 20:
        return "Попробуй переформулировать вопрос. Ответ был слишком короткий: " + answer
    return answer


In [ ]:
# 📦 Установка (если нужно)
# !pip install langchain langchain-openai langgraph

from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from typing import TypedDict

# ✅ Шаг 1: Описание схемы состояния
class MyState(TypedDict):
    question: str
    answer: str

# ✅ Шаг 2: Узел LLM — генерирует ответ
def llm_node(state: MyState) -> MyState:
    llm = ChatOpenAI(model="gpt-4o")
    messages = [
        SystemMessage(content="Ты — эксперт по продукту."),
        HumanMessage(content=f"Вопрос: {state['question']}")
    ]
    response = llm.invoke(messages)
    return {"question": state["question"], "answer": response.content}

# ✅ Шаг 3: Узел уточнения — если ответ слишком короткий
def refine_node(state: MyState) -> MyState:
    return {"question": "Можешь уточнить вопрос?", "answer": "Переформулируй, пожалуйста."}

# ✅ Шаг 4: Условие перехода
def check_quality(state: MyState) -> str:
    if len(state["answer"].split()) < 20:
        return "REFORMULATE"
    return "END"

# ✅ Шаг 5: Построение графа
workflow = StateGraph(state_schema=MyState)
workflow.add_node("ASK_LLM", llm_node)
workflow.add_node("REFORMULATE", refine_node)
workflow.add_node("END", lambda state: state)  # "проходной" узел
workflow.add_conditional_edges("ASK_LLM", check_quality)
workflow.set_entry_point("ASK_LLM")
workflow.set_finish_point("END")

# ✅ Шаг 6: Компиляция и запуск
app = workflow.compile()

# 🔍 Пример использования
result = app.invoke({"question": "Какие у вас тарифы?"})
print(result["answer"])


Извините, но у меня нет доступа к актуальным ценовым предложениям или тарифам. Рекомендую посетить официальный сайт компании или связаться с их службой поддержки для получения самой свежей информации. Если вы уточните, о какой именно услуге или продукте идет речь, я постараюсь предоставить более общую информацию.


In [ ]:

# Пример использования
print(ask_bot("Какие тарифы?"))


Ваш вопрос немного общий, но я постараюсь помочь. Если вы имеете в виду тарифы на какую-либо конкретную услугу или продукт, пожалуйста, уточните, о чем именно идет речь. Вот некоторые примеры, которые могут вас интересовать:

1. **Мобильные компании**: Коммерческие операторы предлагают различные тарифные планы, варьирующиеся по количеству минут, текстовых сообщений и мобильного интернета.

2. **Интернет-провайдеры**: Тарифы различаются по скорости подключения, объему трафика и дополнительным услугам, таким как подключение телевидения.

3. **Страхование**: Тарифы зависят от типа страхования — автострахование, медицинское страхование, страхование имущества и т.д.

4. **Энергообеспечение**: Компания может взимать фиксированную плату за подключение и тарифы на основе объема потребления электроэнергии или газа.

5. **Образование**: Цена за обучение варьируется по уровню (школа, колледж, университет) и престижу учреждения.

Если у вас есть конкретная область или услуга, которая вас интересуе